In [ ]:
import Pkg; Pkg.add(Pkg.PackageSpec(url="https://github.com/JuliaComputing/JuliaAcademyData.jl"))
using JuliaAcademyData; activate("Parallel_Computing")

# Julia is fast

Very often, benchmarks are used to compare languages.  These benchmarks can
lead to long discussions, first as to exactly what is being benchmarked and
secondly what explains the differences.  These simple questions can sometimes
get more complicated than you at first might imagine.

The purpose of this notebook is for you to see a simple benchmark for
yourself.

(This material began life as a wonderful lecture by Steven Johnson at MIT:
[Boxes and registers](https://github.com/stevengj/18S096/blob/master/lectures/lecture1/Boxes-and-registers.ipynb)).

# Outline of this notebook

- Define the sum function
- Implementations & benchmarking of sum in...
    - Julia (built-in)
    - Julia (hand-written)
    - C (hand-written)
    - python (built-in)
    - python (numpy)
    - python (hand-written)
- Towards exploiting parallelism with Julia
    - Allowing for floating point associativity
    - Making use of four cores at once: built-in
    - Making use of four cores at once: hand-written
- Summary of benchmarks

# `sum`: An easy enough function to understand

Consider the  **sum** function `sum(a)`, which computes
$$
\mathrm{sum}(a) = \sum_{i=1}^n a_i,
$$
where $n$ is the length of `a`.

In [1]:
a = rand(10^7) # 1D vector of random numbers, uniform on [0,1)

10000000-element Vector{Float64}:
 0.12170443858416524
 0.05406416678609949
 0.7647346886456676
 0.571280973158476
 0.3540463146060089
 0.6417664725906271
 0.26379448787023074
 0.42128705946223777
 0.1969495230024263
 0.6271605650103085
 0.16470746773721
 0.9104573391417858
 0.8996253349387446
 ⋮
 0.4376777357734525
 0.17382456667783686
 0.16310929075640845
 0.7580473398881458
 0.4954343342805785
 0.9888049683842577
 0.9469044697008451
 0.18591043819707975
 0.10009839359669659
 0.004296624562745821
 0.750145193234373
 0.03041370706436486

In [2]:
sum(a)

4.999722243263498e6

The expected result is ~0.5 * 10^7, since the mean of each entry is 0.5

# Benchmarking a few ways in a few languages

In [3]:
@time sum(a)

  0.003581 seconds (1 allocation: 16 bytes)


4.999722243263498e6

In [4]:
@time sum(a)

  0.003399 seconds (1 allocation: 16 bytes)


4.999722243263498e6

In [5]:
@time sum(a)

  0.003896 seconds (1 allocation: 16 bytes)


4.999722243263498e6

The `@time` macro can yield noisy results, so it's not our best choice for benchmarking!

Luckily, Julia has a `BenchmarkTools.jl` package to make benchmarking easy and accurate:

In [7]:
using Pkg
Pkg.add("BenchmarkTools")
using BenchmarkTools

    Updating registry at `C:\Users\kart-\.julia\registries\General`
    Updating git-repo `https://github.com/JuliaRegistries/General.git`
   Resolving package versions...
    Updating `C:\Users\kart-\.julia\environments\v1.6\Project.toml`
  [6e4b80f9] + BenchmarkTools v1.3.1
    Updating `C:\Users\kart-\.julia\environments\v1.6\Manifest.toml`
  [6e4b80f9] + BenchmarkTools v1.3.1


In [8]:
@benchmark sum($a)

BenchmarkTools.Trial: 1585 samples with 1 evaluation.
 Range (min … max):  2.873 ms …   6.403 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     3.030 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   3.139 ms ± 377.164 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▄█ ▁▅                                                        
  ██▇████▆▇▆▅▄▄▃▃▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▂▂▂▁▂▂▁▂▂▂▂▁▂▂▁▂▁▂▂▂ ▃
  2.87 ms         Histogram: frequency by time        4.75 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

# 1. Julia Built-in

So that's the performance of Julia's built-in sum — but that could be doing any number of tricks to be fast, including not using Julia at all in the first place! Of course, it is indeed written in Julia, but would it perform if we write a naive implementation ourselves?

In [9]:
@which sum(a)

sum(a::AbstractArray; dims, kw...) in Base at reducedim.jl:873

Let's save these benchmark results to a dictionary so we can start keeping track of them and comparing them down the line.

In [10]:
j_bench = @benchmark sum($a)

BenchmarkTools.Trial: 1632 samples with 1 evaluation.
 Range (min … max):  2.877 ms …   4.419 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     2.957 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   3.048 ms ± 202.206 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

   █▆▃                                                         
  ▆████▆▅▅▄▄▃▄▄▄▄▃▄▃▃▃▃▃▃▃▃▃▃▃▃▂▃▃▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▁▂▂▂▂▂▂▂ ▃
  2.88 ms         Histogram: frequency by time         3.8 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [11]:
d = Dict()
d["Julia built-in"] = minimum(j_bench.times) / 1e6
d

Dict{Any, Any} with 1 entry:
  "Julia built-in" => 2.8772

# 2. Julia (hand-written)

In [12]:
function mysum(A)
    s = 0.0
    for a in A
        s += a
    end
    return s
end

mysum (generic function with 1 method)

In [13]:
j_bench_hand = @benchmark mysum($a)

BenchmarkTools.Trial: 643 samples with 1 evaluation.
 Range (min … max):  7.579 ms …   9.281 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     7.743 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   7.768 ms ± 148.200 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

       ▄▅▆▇█▆▃▅▆▆▄▁                                            
  ▂▃▅▇▇██████████████▆▄▄▆▄▄▄▃▃▄▃▁▁▃▂▂▂▃▃▂▁▁▂▂▂▃▁▂▁▁▁▁▂▁▁▁▁▁▁▂ ▄
  7.58 ms         Histogram: frequency by time        8.39 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [14]:
d["Julia hand-written"] = minimum(j_bench_hand.times) / 1e6
d

Dict{Any, Any} with 2 entries:
  "Julia hand-written" => 7.5792
  "Julia built-in"     => 2.8772

So that's about 2x slower than the builtin definition. We'll see why later on.

But first: is this fast?  How would we know?  Let's compare it to some other languages...

#  3. The C language

C is often considered the gold standard: difficult on the human, nice for the machine. Getting within a factor of 2 of C is often satisfying. Nonetheless, even within C, there are many kinds of optimizations possible that a naive C writer may or may not get the advantage of.

The current author does not speak C, so he does not read the cell below, but is happy to know that you can put C code in a Julia session, compile it, and run it. Note that the `"""` wrap a multi-line string.

In [15]:
using Libdl
C_code = """
    #include <stddef.h>
    double c_sum(size_t n, double *X) {
        double s = 0.0;
        for (size_t i = 0; i < n; ++i) {
            s += X[i];
        }
        return s;
    }
"""

const Clib = tempname()   # make a temporary file


# compile to a shared library by piping C_code to gcc
# (works only if you have gcc installed):

open(`gcc -fPIC -O3 -msse3 -xc -shared -o $(Clib * "." * Libdl.dlext) -`, "w") do f
    print(f, C_code)
end

# define a Julia function that calls the C function:
c_sum(X::Array{Float64}) = ccall(("c_sum", Clib), Float64, (Csize_t, Ptr{Float64}), length(X), X)

<stdin>:1:0: warning: -fPIC ignored for target (all code is position independent)
<stdin>: In function 'c_sum':
<stdin>:4:9: error: 'for' loop initial declarations are only allowed in C99 or C11 mode
<stdin>:4:9: note: use option -std=c99, -std=gnu99, -std=c11 or -std=gnu11 to compile your code


LoadError: failed process: Process(`[4mgcc[24m [4m-fPIC[24m [4m-O3[24m [4m-msse3[24m [4m-xc[24m [4m-shared[24m [4m-o[24m [4m'C:\Users\kart-\AppData\Local\Temp\jl_07K1iO9OVz.dll'[24m [4m-[24m`, ProcessExited(1)) [1]


In [ ]:
c_sum(a)

In [ ]:
c_sum(a) ≈ sum(a) # type \approx and then <TAB> to get the ≈ symbolb

We can now benchmark the C code directly from Julia:

In [ ]:
c_bench = @benchmark c_sum($a)

In [ ]:
d["C"] = minimum(c_bench.times) / 1e6  # in milliseconds
d

# 4. Python's built in `sum`

The `PyCall` package provides a Julia interface to Python:

In [16]:
using PyCall

In [17]:
# get the Python built-in "sum" function:
pysum = pybuiltin("sum")

PyObject <built-in function sum>

In [18]:
pysum(a)

4.999722243263187e6

In [19]:
pysum(a) ≈ sum(a)

true

In [20]:
py_list_bench = @benchmark $pysum($a)

BenchmarkTools.Trial: 5 samples with 1 evaluation.
 Range (min … max):  848.516 ms …    1.428 s  ┊ GC (min … max): 19.68% … 28.50%
 Time  (median):     920.220 ms               ┊ GC (median):    24.46%
 Time  (mean ± σ):      1.087 s ± 267.364 ms  ┊ GC (mean ± σ):  25.03% ±  3.78%

  █     ██                                         █          █  
  █▁▁▁▁▁██▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁█ ▁
  849 ms           Histogram: frequency by time          1.43 s <

 Memory estimate: 152.60 MiB, allocs estimate: 10000046.

In [21]:
d["Python built-in"] = minimum(py_list_bench.times) / 1e6
d

Dict{Any, Any} with 3 entries:
  "Julia hand-written" => 7.5792
  "Julia built-in"     => 2.8772
  "Python built-in"    => 848.516

# 5. Python: `numpy`

`numpy` is an optimized C library, callable from Python.
It may be installed within Julia as follows:

In [23]:
Pkg.add("Conda")
using Conda

   Resolving package versions...
    Updating `C:\Users\kart-\.julia\environments\v1.6\Project.toml`
  [8f4d0f93] + Conda v1.7.0
  No Changes to `C:\Users\kart-\.julia\environments\v1.6\Manifest.toml`


In [27]:
Conda.add("numpy")

┌ Info: Running `conda install -y numpy` in root environment
└ @ Conda C:\Users\kart-\.julia\packages\Conda\x2UxR\src\Conda.jl:127


Solving environment: ...working... done

# All requested packages already installed.



In [30]:
numpy_sum = pyimport("numpy")["sum"]

py_numpy_bench = @benchmark $numpy_sum($a)

BenchmarkTools.Trial: 489 samples with 1 evaluation.
 Range (min … max):   9.770 ms …  17.233 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     10.089 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   10.219 ms ± 542.326 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

      ▁▁ ▆▇▇█▃▂▁                                                
  ▂▃▃███████████▇▇▇█▅▅▆▄▃▄▄▄▃▃▁▃▂▃▂▂▂▁▃▃▁▁▃▄▂▂▂▂▁▃▂▁▁▂▂▂▂▁▁▃▁▂ ▃
  9.77 ms         Histogram: frequency by time         11.5 ms <

 Memory estimate: 336 bytes, allocs estimate: 6.

In [31]:
numpy_sum(a)

4.999722243263509e6

In [32]:
numpy_sum(a) ≈ sum(a)

true

In [33]:
d["Python numpy"] = minimum(py_numpy_bench.times) / 1e6
d

Dict{Any, Any} with 4 entries:
  "Python numpy"       => 9.7702
  "Julia hand-written" => 7.5792
  "Julia built-in"     => 2.8772
  "Python built-in"    => 848.516

# 6. Python, hand-written

In [34]:
py"""
def py_sum(A):
    s = 0.0
    for a in A:
        s += a
    return s
"""

sum_py = py"py_sum"

PyObject <function py_sum at 0x0000000001578EE0>

In [35]:
py_hand = @benchmark $sum_py($a)

BenchmarkTools.Trial: 6 samples with 1 evaluation.
 Range (min … max):  969.226 ms … 990.921 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     975.653 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   976.821 ms ±   8.280 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ██     █                    ██                              █  
  ██▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁██▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  969 ms           Histogram: frequency by time          991 ms <

 Memory estimate: 336 bytes, allocs estimate: 6.

In [36]:
sum_py(a)

4.999722243263187e6

In [37]:
sum_py(a) ≈ sum(a)

true

In [38]:
d["Python hand-written"] = minimum(py_hand.times) / 1e6
d

Dict{Any, Any} with 5 entries:
  "Python numpy"        => 9.7702
  "Julia hand-written"  => 7.5792
  "Python hand-written" => 969.226
  "Julia built-in"      => 2.8772
  "Python built-in"     => 848.516

# Summary so far

In [39]:
for (key, value) in sort(collect(d), by=last)
    println(rpad(key, 25, "."), lpad(round(value; digits=1), 6, "."))
end

Julia built-in..............2.9
Julia hand-written..........7.6
Python numpy................9.8
Python built-in...........848.5
Python hand-written.......969.2


We seem to have three different performance classes here: The numpy and Julia
builtins are leading the pack, followed by the hand-written Julia and C
definitions. Those seem to be about 2x slower.  And then we have the much much
slower Python definitions over 100x slower.

# Exploiting parallelism with Julia

The fact that our hand-written Julia solution was almost an even multiple of
2x slower than the builtin solutions is a big clue: perhaps theres some sort
of 2x parallelism going on here?

(In fairness, there are ways to exploit parallelism in other languages, too,
but for brevity we won't cover them)

# 7. Julia (allowing floating point associativity)

The `for` loop

```julia
for a in A
    s += a
end
```

defines a very strict _order_ to the summation: Julia follows exactly what you
wrote and adds the elements of `A` to the result `s` in the order it iterates.
Since floating point numbers aren't associative, a rearrangement here would
change the answer — and Julia is loathe to give you different answer than
the one you asked for.

You can, however, tell Julia to relax that rule and allow for associativity
with the `@fastmath` macro. This might allow Julia to rearrange the sum in an
advantageous manner.

In [40]:
function mysum_fast(A)
    s = 0.0
    for a in A
        @fastmath s += a
    end
    s
end

mysum_fast (generic function with 1 method)

In [41]:
j_bench_hand_fast = @benchmark mysum_fast($a)

BenchmarkTools.Trial: 1718 samples with 1 evaluation.
 Range (min … max):  2.724 ms …   4.528 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     2.793 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.893 ms ± 224.545 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▇█▃                                                         
  ▆███▇▄▄▃▃▃▃▃▃▃▃▄▃▃▃▃▃▃▃▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▂▁▁▁▂▁▂▁▂ ▃
  2.72 ms         Histogram: frequency by time        3.91 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [42]:
mysum_fast(a)

4.999722243263524e6

In [43]:
d["Julia hand-written fast"] = minimum(j_bench_hand_fast.times) / 1e6
d

Dict{Any, Any} with 6 entries:
  "Python numpy"            => 9.7702
  "Julia hand-written"      => 7.5792
  "Python hand-written"     => 969.226
  "Julia built-in"          => 2.8772
  "Python built-in"         => 848.516
  "Julia hand-written fast" => 2.7239

# 8. Distributed Julia (built-in)

We can take this one step further: nearly every modern computer these days has
multiple cores. All the above solutions are working one core hard, but all the
others are just sitting by idly. Let's put them to work!

In [44]:
Pkg.add("DistributedArrays")

   Resolving package versions...
   Installed IntegerMathUtils ── v0.1.0
   Installed Primes ──────────── v0.5.2
   Installed DistributedArrays ─ v0.6.6
    Updating `C:\Users\kart-\.julia\environments\v1.6\Project.toml`
  [aaf54ef3] + DistributedArrays v0.6.6
    Updating `C:\Users\kart-\.julia\environments\v1.6\Manifest.toml`
  [aaf54ef3] + DistributedArrays v0.6.6
  [18e54dd8] + IntegerMathUtils v0.1.0
  [27ebfcd6] + Primes v0.5.2
Precompiling project...
  ✓ IntegerMathUtils
  ✓ Primes
  ✓ DistributedArrays
  3 dependencies successfully precompiled in 2 seconds (89 already precompiled)


In [46]:
using Distributed
using DistributedArrays
addprocs(4)
@sync @everywhere workers() include("/opt/julia-1.0/etc/julia/startup.jl") # Needed just for JuliaBox
@everywhere using DistributedArrays

In [47]:
adist = distribute(a)
j_bench_dist = @benchmark sum($adist)

BenchmarkTools.Trial: 1809 samples with 1 evaluation.
 Range (min … max):  2.309 ms …  11.089 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     2.670 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.744 ms ± 360.196 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

         ▄▃█▇▄▄▄▄▂▂▂ ▂▁                                        
  ▂▁▁▂▄▅█████████████████▇███▇▅▆▆▆▆▆▆▅█▅▆▅▅▅▆▅▅▄▄▅▄▄▄▄▃▃▃▃▂▃▃ ▅
  2.31 ms         Histogram: frequency by time        3.39 ms <

 Memory estimate: 21.11 KiB, allocs estimate: 557.

In [48]:
d["Julia 4x built-in"] = minimum(j_bench_dist.times) / 1e6
d

Dict{Any, Any} with 7 entries:
  "Python numpy"            => 9.7702
  "Julia hand-written"      => 7.5792
  "Python hand-written"     => 969.226
  "Julia built-in"          => 2.8772
  "Python built-in"         => 848.516
  "Julia 4x built-in"       => 2.3087
  "Julia hand-written fast" => 2.7239

# 9. Distributed Julia (hand-written)

Ok, that might be cheating, too — it's again just calling a library
function. Is it possible to write distributed sum ourselves?

In [49]:
function mysum_dist(a::DArray)
    r = Array{Future}(undef, length(procs(a)))
    for (i, id) in enumerate(procs(a))
        r[i] = @spawnat id sum(localpart(a))
    end
    return sum(fetch.(r))
end

mysum_dist (generic function with 1 method)

In [50]:
j_bench_hand_dist = @benchmark mysum_dist($adist)

BenchmarkTools.Trial: 1389 samples with 1 evaluation.
 Range (min … max):  3.015 ms …   4.497 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     3.550 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   3.585 ms ± 240.826 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

                   ▅█▅▄▄▂▃▅ ▃▂    ▁ ▁  ▁                       
  ▁▁▁▂▂▂▁▂▄▄▄▅▄▄▇██████████▇███▆▇████▆███▇▆▅▇▄▇▅▄▃▄▃▃▂▃▂▂▂▂▂▁ ▄
  3.02 ms         Histogram: frequency by time         4.2 ms <

 Memory estimate: 34.09 KiB, allocs estimate: 883.

In [51]:
d["Julia 4x hand-written"] = minimum(j_bench_hand_dist.times) / 1e6
d

Dict{Any, Any} with 8 entries:
  "Python numpy"            => 9.7702
  "Julia hand-written"      => 7.5792
  "Python hand-written"     => 969.226
  "Julia built-in"          => 2.8772
  "Python built-in"         => 848.516
  "Julia 4x built-in"       => 2.3087
  "Julia 4x hand-written"   => 3.0155
  "Julia hand-written fast" => 2.7239

# Overall Summary

In [52]:
for (key, value) in sort(collect(d), by=last)
    println(rpad(key, 25, "."), lpad(round(value; digits=1), 6, "."))
end

Julia 4x built-in...........2.3
Julia hand-written fast.....2.7
Julia built-in..............2.9
Julia 4x hand-written.......3.0
Julia hand-written..........7.6
Python numpy................9.8
Python built-in...........848.5
Python hand-written.......969.2


# Key take-aways

* Julia allows for serial C-like performance, even with hand-written functions
* Julia allows us to exploit many forms of parallelism to further improve performance. We demonstrated:
    * Single-processor parallelism with SIMD
    * Multi-process parallelism with DistributedArrays
* But there are many other ways to express parallelism, too!